In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import shutil



In [2]:
#STEP 1 Construction of label map
def construct_label_map(image_dir):
    label_map = {}
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            # Extract label from filename (assuming label is before the first digit)
            label = ''.join(filter(str.isalpha, filename.split('.')[0]))
            label_map[filename] = label
    return label_map

# Directory paths for each label
label_directories = {
    'light': '/Users/jaysungavillan/Documents/DesertRose/extracted_photos/light',
    'double': '/Users/jaysungavillan/Documents/DesertRose/extracted_photos/double',
    'petite': '/Users/jaysungavillan/Documents/DesertRose/extracted_photos/petite',
    'standard': '/Users/jaysungavillan/Documents/DesertRose/extracted_photos/standard',
}

# Construct label map for each label directory
full_label_map = {}
for label, directory in label_directories.items():
    label_map = construct_label_map(directory)
    full_label_map.update(label_map)

##print(full_label_map)

In [3]:
# Step 2: Manually label the images

# image_dir = directory to the path where the images are stored
# Function to move images to corresponding label directories
def label_images(image_dir, label):
    # Iterate over images directly in the image directory
    for filename in os.listdir(image_dir):
        if filename.endswith('.jpg'):
            src_path = os.path.join(image_dir, filename)
            label = full_label_map[filename]  # Get label from label_map based on filename
            dest_path = os.path.join('/Users/jaysungavillan/Documents/DesertRose/labeled', label, filename)
            shutil.copyfile(src_path, dest_path)

# Manually label images
label_images('/Users/jaysungavillan/Documents/DesertRose/extracted_photos/light', 'light')
label_images('/Users/jaysungavillan/Documents/DesertRose/extracted_photos/double', 'double')
label_images('/Users/jaysungavillan/Documents/DesertRose/extracted_photos/petite', 'petite')
label_images('/Users/jaysungavillan/Documents/DesertRose/extracted_photos/standard', 'standard')


In [ ]:
# Step 3: Train a classifier

# image_dir = directory to the path where the labled images are stored
# Function to extract features and labels from labeled images
def extract_features(image_dir):
    features = []
    labels = []

    for label, dir_path in full_label_map.items():
        # Iterate over subdirectories (labels) within dir_path
        for subdir_label in os.listdir(image_dir):
            subdir_path = os.path.join(image_dir, subdir_label)
            if os.path.isdir(subdir_path):
                for filename in os.listdir(subdir_path):
                    if filename.endswith('.jpg'):
                        img_path = os.path.join(subdir_path, filename)
                        img = image.load_img(img_path, target_size=(224, 224))
                        img_array = image.img_to_array(img)
                        img_array = np.expand_dims(img_array, axis=0)
                        img_array = preprocess_input(img_array)

                        # Extract features using ResNet50 model
                        features.append(img_array)
                        labels.append(label)

    return np.vstack(features), labels
# Extract features and labels
features, labels = extract_features('/Users/jaysungavillan/Documents/DesertRose/labeled')

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Step 3: Train a classifier

# Dummy classifier (using file paths as features)
class DummyClassifier:
    def fit(self, X_train, y_train):
        pass

    def predict(self, X_test):
        return X_test  # Predict using file paths

# Train the classifier
classifier = DummyClassifier()
classifier.fit(X_train, y_train)

# Step 4: Evaluate the classifier

# Predict labels for the test set
y_pred = classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Create confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=label_map.keys(), yticklabels=label_map.keys())
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()